In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [3]:
text = open('input.txt', 'r').read()

In [4]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)
''.join(chars), vocab_size

("\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz", 65)

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [7]:
encode('Hi, this is Mrigank')

[20, 47, 6, 1, 58, 46, 47, 57, 1, 47, 57, 1, 25, 56, 47, 45, 39, 52, 49]

In [8]:
decode(encode('Hi, this is Mrigank'))

'Hi, this is Mrigank'

In [9]:
# Let's encode entire text dataset and store in torch.Tensor

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# split the data into train & val

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

## We'll never feed entire text to the Transformer all at once:
- Coz it'll be just very inefficient

## We only work with chunks
- "max_length"

In [11]:
block_size = 8

In [12]:
train_data[:block_size+1]
# this will have multiple examples packed into it; actually 8 individual examples exist in this

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target: {target}")

when input is tensor([18]), the target: 47
when input is tensor([18, 47]), the target: 56
when input is tensor([18, 47, 56]), the target: 57
when input is tensor([18, 47, 56, 57]), the target: 58
when input is tensor([18, 47, 56, 57, 58]), the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target: 58


## Using inputs of varying length as shown above:
- Transformer would learn to generate on input size as small as 1, upto ```block_size```
- This would be needed while inferring: we need to start sampling as soon as input of size 1 occurs

In [14]:
torch.randint(1000 - 8, (4,))

tensor([119, 925, 796, 360])

In [15]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 # what is the max context length for predictions?

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)
print('--------------------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        targets = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------------------------
when input is [24] the target: 58
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 58
when input is [24, 43, 58, 5] the target: 58
when input is [24, 43, 58, 5, 57] the target: 58
when input is [24, 43, 58, 5, 57, 1] the target: 58
when input is [24, 43, 58, 5, 57, 1, 46] the target: 58
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 58
when input is [44] the target: 58
when input is [44, 53] the target: 58
when input is [44, 53, 56] the target: 58
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the targe

### Above - we have 4x8 array which is basically a batch.
- But, the inputs going into the NN are 32 (8 inputs from each item (there are 4 items) in a batch)

## Let's start giving this batch of data to NNs,
- We'll start with simplest NN for LM: BigramLM

In [19]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

- ```nn.Embedding``` is a lookup tensor, which gives gives embeddings for indices of its input
- https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#embedding

In [46]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off thee logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C): B-Batch(4); T-Time steps(8); C-channel(65)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            # These 2 reshapes are being done to adjust logits & targets dims to Torch's cross-entropy function
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on the last time step: coz it's predicting based on 1 last char
            logits = logits[:, -1, :] # (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [54]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.7125, grad_fn=<NllLossBackward0>)


In [55]:
# Generation starting with new-line character

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


AU.-
AsXmkD'vxMdzvseJuFowr&LJo
Jm,o;,ACILjp!t3PjUqWJZphJmyuQgYARv:rCFXm.h$$PKYsx
PTfCIta,UXDCwCIozXD


### Above generation is garbage right now, let's improve it

In [56]:
# create a pytorch optimizer object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [57]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb,yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.5586419105529785


In [58]:
# Generate after training
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


LI'd ard yois lld-N:
I y?
weiorof y, I d far hith w, prvel, d ke,

'Pa thend sous uine ased il ouf g
